In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
# import signal_generation as sg
# import channel_model as ch
import hpcom
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
# from sklearn.externals import joblib
from datetime import datetime
from importlib import reload
reload(hpcom)

from composer import *
from estimator import *

In [ ]:
df = pd.read_pickle('data/data_collected_3.pkl')

n_neighbours = 4
n_gauss = 5
# train_runs = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
train_runs = [0, 1, 2, 3, 4, 5]
test_runs = [14, 15]

p_ave_dbm = 6
z_km = 80 * 20
n_channels = 1
sigma = 1.5

df_tree = get_data_with_m_neighbor(df, z_km, p_ave_dbm, n_channels, train_runs, n_neighbours, r=0.1, n_gauss=n_gauss)
df_tree_for_test = get_data_with_m_neighbor(df, z_km, p_ave_dbm, n_channels, test_runs, n_neighbours, r=0.1, n_gauss=n_gauss)

# df_tree = get_data_gauss_parallel(df, z_km, p_ave_dbm, n_channels, train_runs, n_neighbours, r=0.1, n_gauss=n_gauss, sigma=sigma)
# df_tree_for_test = get_data_gauss_parallel(df, z_km, p_ave_dbm, n_channels, test_runs, n_neighbours, r=0.1, n_gauss=1, sigma=sigma)

print(df_tree.head())
df = pd.DataFrame([])
del df

In [ ]:
labels_to_drop = ['point_orig_abs', 'point_orig_angle',
                  'point_label',
                  'diff_real', 'diff_imag',
                  'diff_sq_real', 'diff_sq_imag',
                  'diff_circle_real', 'diff_circle_imag',
                  'diff_p3_real', 'diff_p3_imag']

# for get_data_with_m_neighbor
labels_to_drop = labels_to_drop + [f'diff_gauss_{m}_real' for m in range(n_gauss)] + [f'diff_gauss_{m}_imag' for m in range(n_gauss)]

# for get_data_gauss_parallel
# labels_to_drop = labels_to_drop + [f'diff_gauss_real'] + [f'diff_gauss_imag']

# chose what to predict
# label_predict = ['diff_circle_real', 'diff_circle_imag']
label_predict = ['diff_real', 'diff_imag']
m = 5
# label_predict = [f'diff_gauss_{m}_real', f'diff_gauss_{m}_imag']
# label_predict = [f'diff_gauss_real', f'diff_gauss_imag']
# label_predict = ['diff_p3_real', 'diff_p3_imag']
# label_predict = ['point_label']

In [ ]:
X_for_test = df_tree_for_test.drop(labels=labels_to_drop, axis=1)
points_init_for_test = df_tree_for_test['point_orig_abs'].values * np.exp(1.0j * df_tree_for_test['point_orig_angle'])
labels_for_test = df_tree_for_test['point_label']

In [ ]:
y = df_tree[[label_predict[0], label_predict[1]]].values
X = df_tree.drop(labels=labels_to_drop, axis=1)

In [ ]:
split_ratio = 0.01
random_state = 42

# X_train, X_test, y_real_train, y_real_test = train_test_split(X, y_real, test_size=split_ratio, random_state=random_state)
# _, _, y_imag_train, y_imag_test = train_test_split(X, y_imag, test_size=split_ratio, random_state=random_state)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_ratio, random_state=random_state)
po_abs_train, po_abs_test, po_angle_train, po_angle_test = train_test_split(df_tree['point_orig_abs'].values, df_tree['point_orig_angle'].values, test_size=split_ratio, random_state=random_state)

In [ ]:
n_estimators = 300
random_state_for_tree = 73
start_time = datetime.now()
model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state_for_tree, n_jobs=13, verbose=1)
model.warm_start = True  # start from previous state of the forest (add new trees)
# model.n_estimators += n_estimators
model.fit(X_train, y_train)
print("model real took", (datetime.now() - start_time).total_seconds() * 1000, "ms")